# Obtencion de datos


El objetivo es encotrar la informacion mas posible, y que sea de calidad. De acuerdo a los criterios de PLD.
El sistema/base de datos es TAS (SISTEMA DE MERCADO DE DINERO Y CAMBIOS) el cual registra y controla y administra las operaciones que realizan el area de Mercado de Dinero con clientes e intermediarios, refente a comprar y ventas de los diferentes instrumentos del mercado , asi como el registro, control y administracion las operaciones.

Para poder describir la informacion de la base de datos, se debe conocer ¿Quien es el cliente?

| Información requerida                   | Información obtenida                              |
|-----------------------------------------|--------------------------------------------------|
| negocio                                 |                                                  |
| nombre cliente o razón social           |          "U##NOMBRE", NOMBRE, NOMLARGO    |
| número cliente                          |          CUENTA                               |
| estatus                                 |         FBAJA IS NULL                            |
| número contrato                         |                                     |
| género                                  |                      N/A                            |
| fecha_nacimiento o de constitución      |                                                  |
| entidad federativa nacimiento           |                                                  |
| país de nacimiento                      |    if INACIONALIDAD == 1 then "MEXICO" else "No se cuenta con la informacion"                                                |
| nacionalidad                            |   INACIONALIDAD  esta columna proviene de un catálogo de la tabla TAS.FNACIONALIDAD                             |
| profesión                               |                    IACTIV_ECONO                              |
| calle                                   |                                 |
| número_exterior                         |                                                  |
| número_interior                         |                                                  |
| colonia_urbanización                    |                                                  |
| delegación_municipio                    |                                                  |
| ciudad_población                        |                                                  |
| entidad_federativa                      |                                    |
| código postal                           |                                                  |
| país                                    |                                |
| teléfono                                |    TELEFONO1, TELEFONO2                                |
| correo_electrónico                      |                    EMAIL                              |
| rfc                                     |  RFC   |
| curp                                    |  CURP                                                |
| fiel                                    |                                                  |
| representante legal                     |   Esta informacion esta en la tabla TAS.FAPODERADO se  debe hacer un join                                                |
| tipo de persona                         |  TIPINVE esta columa proviene de un catálogo de la tabla TAS.TIPINVER  |
| clasificación por grado de riesgo       |                                                  |
| pep (persona políticamente expuesta)    |                                                  |
| razón                                   |                                                  |
| producto contratado                     |                   |
| cambios en perfil transaccional         |                                                  |


- Esta informacion no se puede obtener de la base de datos, ya que no se cuenta con la descripcion de requerimiento

    - clasificacion por grado de riesgo	
    - razon
    - cambios en perfil transaccional 

In [1]:
query = """
WITH CLIENTES_ACTIVOS AS(
	SELECT * 
    FROM TAS.CLIENTE cl
    WHERE cl.FBAJA IS NULL
), 
CLIENTES_CON_NA AS(
	SELECT CA.*, FN.NNACIONALIDAD NACIONALIDAD
	FROM CLIENTES_ACTIVOS CA
	INNER JOIN TAS.FNACIONALIDAD FN ON CA.INACIONALIDAD = FN.INACIONALIDAD
), 
CLIENTES_CON_TRANS_2024 AS (
	SELECT CCN.*, ft.FOPER, ft.FLIQ, ft.IORDEN, ft.ITINSTR ,ft.IINSTR, 
			ft."U##IEMISION", ft.ISERIE, ft.MBRUTO, ft.MCOSTO, ft.NTITULOS
	FROM CLIENTES_CON_NA CCN
	LEFT JOIN TAS.FTRANS ft ON CCN.CUENTA = ft.ICONTRATO
	WHERE FOPER > TO_DATE('2024-01-01', 'YYYY-MM-DD')
), 
CLIENTES_CON_TIP_INV AS (
	SELECT CCT.*, tp.DESCRIP TIPO_DE_PERSONA
	FROM CLIENTES_CON_TRANS_2024 CCT
	LEFT JOIN TAS.TIPINVER tp ON CCT.TIPINVE = tp.TIPINVE
) SELECT 
	'' NEGOCIO, 
	'' NOMBRE_O_RAZON_SOCIAL,
	"U##NOMBRE" , NOMBRE, NOMLARGO,
	CUENTA, 
	'VIGENTE' ESTATUS, 
	'' NUMERO_CONTRATO,
	'' GENERO,  
	'' FECHA_NAC_O_CONST, 
	'' ENTIDAD_FEDERATIVA_NACIMIENTO, 
	'' PAIS_DE_NACIMIENTO, 
	NACIONALIDAD,
	IACTIV_ECONO PROFESION,
	'' CALLE,
	'' NUMERO_EXTERIOR, 
	'' NUMERO_INTERIOR, 
	'' COLONIA_URBANIZACION, 
	'' DELEGACION_MUNICIPIO, 
	'' CIUDAD_POBLACION,  
	'' ENTIDAD_FEDERATIVA, 
	'' CODIGO_POSTAL, 
	IREGIONAL PAIS,
	'' TELEFONO, 
	TELEFONO1 , TELEFONO2,
	EMAIL CORREO_ELECTRONICO, 
	RFC RFC, 
	'' CURP, 
	'' FIEL,
	'' REPRESENTANTE_LEGAL,
	TIPO_DE_PERSONA TIPO_PERSONA, 
	'' PRODUCTO_CONTRATADO, 
	TIPINVE, FOPER, FLIQ, IORDEN, ITINSTR , IINSTR, "U##IEMISION", ISERIE, MBRUTO, MCOSTO, NTITULOS, 
	'TAS' SISTEMA_ORIGEN
FROM CLIENTES_CON_TIP_INV
"""

query2 = "SELECT CUENTA NUMERO_CLIENTE, APE_PAT || ' ' || APE_MAT || ' ' || NOMBRE AS REPRESENTANTE_LEGAL FROM TAS.FAPODERADO"

In [2]:
from utils.tf import *
from utils.db import *
from utils.gd import * 
from utils.clt_repetidos import *
from datetime import datetime
import cx_Oracle
import pandas as pd
import os
from rapidfuzz import process, fuzz
import numpy as np

In [3]:
tas_clientes = pd.read_csv("/home/ale1726/proyects/datalake/clientes/data/Clientes_activos_TAS.dat")

In [109]:
def get_data_tas(database,path_exit, query, query2):
    conn = cx_Oracle.connect(user=database['USER'], password=database["PSSWD"], dsn=database["DSN"])
    cursor = conn.cursor()
    
    cursor.execute(query)
    columnas_clientes = [columna[0] for columna in cursor.description]
    tas_clientes = pd.DataFrame(cursor.fetchall(), columns=columnas_clientes)
    
    cursor.execute(query2)
    columnas_fapoderado = [columna[0] for columna in cursor.description]
    fapoderado =   pd.DataFrame(cursor.fetchall(), columns=columnas_fapoderado)

    representa_legal = fapoderado.groupby('CUENTA')['REPRESENTANTE_LEGAL'].apply(lambda x: ', '.join(x)).reset_index()
    
    resultado = pd.merge(tas_clientes, representa_legal, on="CUENTA", how="left")
    resultado.to_csv(os.path.join(path_exit,"Transformacion_clientes_TAS.dat"),index=False)
    
    cursor.close()
    conn.close()
    

In [4]:
conn = cx_Oracle.connect(user=db_tas['USER'], password=db_tas["PSSWD"], dsn=db_tas["DSN"])
cursor = conn.cursor()
cursor.execute(query2)
columnas_fapoderado = [columna[0] for columna in cursor.description]
fapoderado =   pd.DataFrame(cursor.fetchall(), columns=columnas_fapoderado)

In [5]:
representa_legal = fapoderado.groupby('NUMERO_CLIENTE')['REPRESENTANTE_LEGAL'].apply(lambda x: ', '.join(x)).reset_index()

In [6]:
df = pd.merge(tas_clientes, representa_legal, on="NUMERO_CLIENTE", how="left")

In [7]:
df["NOMBRE_O_RAZON_SOCIAL"] = df["NOMLARGO"]

In [8]:
df["REPRESENTANTE_LEGAL"] = df["REPRESENTANTE_LEGAL_y"]

In [9]:
df["REPRESENTANTE_LEGAL"] = df["REPRESENTANTE_LEGAL"].apply(lambda x: " -- ".join(sorted(set(c.strip() for c in x.split(",") if c.strip() != 'NA' and c.strip()  != "Datos de la Escritura"))) if "," in str(x) else x)

In [10]:
cursor.close()
conn.close()

In [11]:
df["RFC"] = df["RFC"].apply(lambda x: normalizar_RFC(x) if pd.notna(x) and verificar_rfc(normalizar_RFC(x)) else "XXXXXXXXXXXXX")

In [12]:
def limpiar_telefono(col1, col2):
    if col1 == col2:
        return col1
    elif not col1.strip() and col2:
        return col2
    elif col1 and not col2.strip():
        return col1
    elif col1.strip() and col2.strip() and col1 != col2:
        return f"{col1}, {col2}"
    else:
        return ""

In [13]:
df["TELEFONO"] = df.apply(lambda row:( limpiar_telefono(row["TELEFONO1"],row["TELEFONO2"])
                            ),axis=1)

In [14]:
df['FECHA_DE_ACTUALIZACION'] = datetime.now().strftime('%d/%m/%y')

In [15]:
col_final = ['NEGOCIO', 'NOMBRE_O_RAZON_SOCIAL', 'NUMERO_CLIENTE', 'ESTATUS',
       'NUMERO_CONTRATO', 'GENERO', 'FECHA_NAC_O_CONST',
       'ENTIDAD_FEDERATIVA_NACIMIENTO', 'PAIS_DE_NACIMIENTO', 'NACIONALIDAD',
       'PROFESION', 'CALLE', 'NUMERO_EXTERIOR', 'NUMERO_INTERIOR',
       'COLONIA_URBANIZACION', 'DELEGACION_MUNICIPIO', 'CIUDAD_POBLACION',
       'ENTIDAD_FEDERATIVA', 'CODIGO_POSTAL', 'PAIS', 'TELEFONO',
       'CORREO_ELECTRONICO', 'RFC', 'CURP', 'FIEL', 'REPRESENTANTE_LEGAL',
       'TIPO_PERSONA', 'PRODUCTO_CONTRATADO',
       'SISTEMA_ORIGEN', 'FECHA_DE_ACTUALIZACION']
df = df[col_final]

In [31]:
def normalizar_cadena(cadena):
    # Eliminación de caracteres especiales
    cadena_normalizada = re.sub(r'[^a-zA-Z0-9áéíóúÁÉÍÓÚ\s]', '', cadena)
    # Conversión a mayúsculas
    cadena_normalizada = cadena_normalizada.upper()
    # Eliminación de espacios en blanco
    cadena_normalizada = cadena_normalizada.replace(' ', '')
    return cadena_normalizada

def compara_filas(args):
    id1, num_cliente_1, clt1, df2, col_nom_2, id_cliente2, criterio_similitud = args
    resultados = [[], [], [], [], [], []]
    for id2, num_cliente_2, clt2 in zip(df2.index,df2[id_cliente2], df2[col_nom_2]):
        if fuzz.ratio(normalizar_cadena(clt1), normalizar_cadena(clt2)) >= criterio_similitud:
            resultados[0].append(clt1)
            resultados[1].append(clt2)
            resultados[2].append(id1)
            resultados[3].append(id2)
            resultados[4].append(num_cliente_1)
            resultados[5].append(num_cliente_2)
    return resultados

def find_transition(lista):
    for p1 in range(len(lista)):
        for p2 in range(p1 + 1, len(lista)):
            #print(lista[p1], lista[p2])
            if lista[p1][0] == lista[p2][0]:
                #print("paso 1")
                if (lista[p1][1], lista[p2][1]) in lista:
                    return (lista[p1][1], lista[p2][1])
                elif (lista[p2][1], lista[p1][1]) in lista:
                    return (lista[p2][1], lista[p1][1])
            elif lista[p1][1] == lista[p2][0]:
                #print("paso 2")
                if (lista[p1][0], lista[p2][1]) in lista:
                     return (lista[p1][0], lista[p2][1])
                elif (lista[p2][1], lista[p1][0]) in lista:
                    return (lista[p2][1], lista[p1][0])
            elif lista[p1][0] == lista[p2][1]:
                #print("paso 3")
                if (lista[p1][1], lista[p2][0]) in lista:
                    return (lista[p1][1], lista[p2][0])
                elif (lista[p2][0], lista[p1][1]) in lista:
                    return (lista[p2][0], lista[p1][1])
            elif lista[p1][1] == lista[p2][1]:
                #print("paso 4")
                if (lista[p1][0], lista[p2][0]) in lista:
                    return (lista[p1][0], lista[p2][0])
                elif (lista[p2][0], lista[p1][0]) in lista:
                    return (lista[p2][0], lista[p1][0])
    return []

def clean_transition_aux(lista):
    uz = find_transition(lista)
    if uz:
        lista.remove(uz)
    while uz:
        uz = find_transition(lista)
        if uz:
            lista.remove(uz)
    return lista

def clean_simetria(df, identificador1:str, identificador2:str):
    """
    Genera un dataframe con un estimado de almacenamiento de los sistemas dados, exporta u envia a una base de datos.
    
    Args:
        idx (list): Lista de tuplas con las posibilidades de combinacion 
    Returns:
        pd.DataFrame: DataFrame concatenado con los metadatos de los sistemas.
        pd.DataFrame: DataFrame transformado con los detalles del almacenamiento promedio.
        pd.DataFrame: DataFrame transformado con los detalles del almacenamiento.
    """
    idx = [((id1,id2),(id2,id1)) for id1, id2 in zip(df[identificador1].to_list(), df[identificador2].to_list())]
    
    for i in idx:
        if (i[1], i[0]) in idx:
            idx.remove((i[1], i[0]))
  
    ix = [df.loc[(df[identificador1] == i[0][0]) & (df[identificador2] == i[0][1])].index for i in idx]

    ix = [x[0] for x in ix]
    df = df.loc[ix]
    return df


def clean_transition(df, identificador1:str, identificador2:str):
    parejas = [(id1,id2) for id1, id2 in zip(df[identificador1].to_list(),df[identificador2].to_list())]
    idx_clean =  clean_transition_aux(parejas)
    ix_2 = [df.loc[(df[identificador1] == i[0]) & (df[identificador2] == i[1])].index for i in idx_clean]
    ix_2  = [x[0] for x in ix_2]
    clean_df = df.loc[ix_2]
    return clean_df
    

def compara_nombres(df1, df2, col_nom_1, col_nom_2, id_cliente1, id_cliente2, criterio_similitud):
    """
    Compara los nombres de dos dataframe mediante un umbral de similitud.

    Args:
        df1 (pd.DataFrame): Primera tabla de clientes.
        df2 (pd.DataFrame): Segunda tabla de clientes.
        col_nom_1 (str): nombre de la columna que contiene los nombres del df1.
        col_nom_2 (str): nombre de la columna que contiene los nombres del df2.
        id_cliente1(str): id del cliente en la primera tabla.
        id_cliente2(str): id del cliente en la segunda tabla.     
        criterio_similitud (int): criterio de similitud.

    Returns:
        list: Listas de resultados con las coincidencias encontradas.
    """ 
    CR = [[], [], [], [], [], []]

    with Pool(processes=10) as pool:
        args = [(id1, num_cliente_1, clt1, df2, col_nom_2, id_cliente2, criterio_similitud) for id1, num_cliente_1, clt1 in zip(df1.index, df1[id_cliente1], df1[col_nom_1])]
        resultados = pool.map(compara_filas, args)
    
    for resultado in resultados:
        for i in range(6):
            CR[i].extend(resultado[i])
            
    df_comparacion = pd.DataFrame(
        {   "ID1": CR[2],
            "NUM_CLIENTE_1": CR[4],
            "CLIENTE_1" : CR[0],
            "ID2": CR[3],
            "NUM_CLIENTE_2": CR[5],
            "CLIENTE_2" : CR[1]
        }
    )
    df_aux = df_comparacion[df_comparacion["NUM_CLIENTE_1"]!=df_comparacion["NUM_CLIENTE_2"]]
    df_aux = clean_simetria(df_aux,"NUM_CLIENTE_1","NUM_CLIENTE_2")
    df_aux = clean_transition(df_aux,"NUM_CLIENTE_1","NUM_CLIENTE_2")
    return df_aux

In [36]:
clt_repetidos = compara_nombres(df, df, "NOMBRE_O_RAZON_SOCIAL", "NOMBRE_O_RAZON_SOCIAL","NUMERO_CLIENTE","NUMERO_CLIENTE", 99)

In [37]:
clt_repetidos

,ID1,NUM_CLIENTE_1,CLIENTE_1,ID2,NUM_CLIENTE_2,CLIENTE_2
145,1901,"1064055, 1057536","BANCO INVEX, S.A. INSTITUCION DE BANCA MULTIPL...",462,1059975,"BANCO INVEX, S.A., INSTITUCION DE BANCA MULTIP..."
387,1462,1062036,DIRECCION DE FIDEICOMISOS 80174 FDO P/DESARROL...,1616,1060243,DIRECCION DE FIDEICOMISOS 80174 FDO. DESARROLL...
981,1958,"1064277, 1064238, 1064230, 1063932, 1063913",NACIONAL FINANCIERA SNC EN SU CARACTER DE FIDU...,1954,"1064395, 1064394, 1064393, 1064392, 1064391, 1...",NACIONAL FINANCIERA SNC EN SU CARACTER DE FIDU...
982,1958,"1064277, 1064238, 1064230, 1063932, 1063913",NACIONAL FINANCIERA SNC EN SU CARACTER DE FIDU...,1971,"1063936, 1063949",NACIONAL FINANCIERA SNC EN SU CARACTER DE FIDU...
983,1958,"1064277, 1064238, 1064230, 1063932, 1063913",NACIONAL FINANCIERA SNC EN SU CARACTER DE FIDU...,1902,"1064016, 1064264, 1064047, 1064046, 1063907, 1...","NACIONAL FINANCIERA, SNC EN SU CARACTER DE FID..."
992,1957,"1064330, 1064320, 1064080, 6009238, 1064244, 1...",NACIONAL FINANCIERA SNC EN SU CARACTER DE FIDU...,1904,"1063904, 1064083, 1064483, 1064482, 1064481, 1...",NACIONAL FINANCIERA SNC EN SU CARACTER DE FIDU...
993,1957,"1064330, 1064320, 1064080, 6009238, 1064244, 1...",NACIONAL FINANCIERA SNC EN SU CARACTER DE FIDU...,1132,1063905,NACIONAL FINANCIERA SNC EN SUC ARACTER DE FIDU...
994,1957,"1064330, 1064320, 1064080, 6009238, 1064244, 1...",NACIONAL FINANCIERA SNC EN SU CARACTER DE FIDU...,1902,"1064016, 1064264, 1064047, 1064046, 1063907, 1...","NACIONAL FINANCIERA, SNC EN SU CARACTER DE FID..."
1000,1900,"1064225, 1064278",NACIONAL FINANCIERA SNC EN SU CARACTER DE FIDU...,1903,"1064228, 1064422, 1064235","NACIONAL FINANCIERA, SNC EN SU CARACTER DE FID..."
1004,1275,1064351,NACIONAL FINANCIERA SNC EN SU CARACTER DE FIDU...,1898,"1064324, 1064119","NACIONAL FINANCIERA, SNC EN SU CARACTER DE FID..."


In [34]:
if not clt_repetidos.empty:
    df_resul, drops_ids = agregar_asociaciones_clientes(clt_repetidos,df)
    df = df.drop(drops_ids)
    df = pd.concat([df, df_resul], ignore_index=True)
    df = df.sort_values("NOMBRE_O_RAZON_SOCIAL")

/home/ale1726/proyects/datalake/clientes/clientes_eda/utils/clt_repetidos.py:183: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_resul = df_resul.replace("-", np.nan)


In [35]:
name_archivo = os.path.join("/home/ale1726/proyects/datalake/clientes/data/ETL","CLientesT_TAS.csv")
df.to_csv(name_archivo,index=False)

In [110]:
path_exit = "/home/ale1726/proyects/datalake/clientes/data"


In [113]:
get_data_tas(db_tas, path_exit, query, query2)

In [2]:
df = pd.read_csv("/home/ale1726/proyects/datalake/clientes/data/Transformacion_clientes_TAS.dat")

In [ ]:
df

In [103]:
df_fitlred = df[df["CUENTA"] == 76000]

In [104]:
df_cts_pld = pd.read_csv("/home/ale1726/proyects/dash/dash-anlytc_data/data/Acumulado_clientes.csv")

In [106]:
df_cts_pld.value_counts("producto contratado")

producto contratado
Contrato de Depósito Bancario de Títulos en Administración y de Comisión Mercantil No Discrecional                                                                          1254
Contrato Marco Para Operaciones de Compraventa de Valores y Reporto                                                                                                          697
FIDEICOMISO                                                                                                                                                                  539
Línea de Crédito                                                                                                                                                              95
Programa Primer Piso                                                                                                                                                          57
Contrato de Depósito y Garantía Prendaria                                                      

In [105]:
df_cts_pld.columns

Index(['negocio', 'nombre cliente o razon social', 'numero cliente', 'estatus',
       'numero contrato', 'genero', 'fecha_nacimiento o de constitucion',
       'entidad federativa nacimiento', 'pais de nacimiento', 'nacionalidad',
       'profesion', 'calle', 'numero_exterior', 'numero_interior',
       'colonia_urbanizacion', 'delegacion_municipio', 'ciudad_poblacion',
       'entidad_federativa', 'codigopostal', 'pais', 'telefono',
       'correo_electronico', 'rfc', 'curp', 'fiel', 'representante legal',
       'tipo de persona', 'clasificacion por grado de riesgo',
       'pep (persona políticamente expuesta)', 'razon', 'producto contratado',
       'cambios en perfil transaccional'],
      dtype='object')

In [ ]:
df_fitlred

In [ ]:
df.head()

,CUENTA,U##NOMBRE,NOMBRE,NOMLARGO,ESTATUS,NUMERO DE CONTRATO,INACIONALIDAD,IACTIV_ECONO,TELEFONO1,TELEFONO2,...,FLIQ,IORDEN,ITINSTR,IINSTR,U##IEMISION,ISERIE,MBRUTO,MCOSTO,NTITULOS,REPRESENTANTE_LEGAL
0,76000,SAGAPL,SAGAPL,SAGAPL,1,76000,1,BANCA DE DESARROLLO,,,...,2024-10-21 00:00:00,136612,GUBERR,BPA182,BPA182,IS300328,5.000000e+09,5.189662e+09,51322766,NaN
1,76000,SAGAPL,SAGAPL,SAGAPL,1,76000,1,BANCA DE DESARROLLO,,,...,2024-10-21 00:00:00,136611,GUBERR,BPA182,BPA182,IS300328,5.189662e+09,5.189662e+09,51322766,NaN
2,76000,SAGAPL,SAGAPL,SAGAPL,1,76000,1,BANCA DE DESARROLLO,,,...,2024-10-21 00:00:00,136610,GUBERR,BONDESF,BONDESF,LF251023,5.000000e+09,5.082295e+09,50497249,NaN
3,76000,SAGAPL,SAGAPL,SAGAPL,1,76000,1,BANCA DE DESARROLLO,,,...,2024-10-21 00:00:00,136609,GUBERR,BONDESF,BONDESF,LF251023,5.082295e+09,5.082295e+09,50497249,NaN
4,76000,SAGAPL,SAGAPL,SAGAPL,1,76000,1,BANCA DE DESARROLLO,,,...,2024-10-18 00:00:00,135801,GUBERR,BPAG91,BPAG91,IQ270506,5.000000e+09,5.080935e+09,49963230,NaN


In [ ]:
r1 = len(df[df["U##NOMBRE"].isna()])
r2 = len(df[df["NOMBRE"].isna()])
r3 = len(df[df["NOMBRE"].isna()])
print(r1,r2,r3)

0 0 0


In [83]:
df_filtred = df.drop_duplicates("NOMLARGO")
df_filtred = df_filtred[["U##NOMBRE", "NOMBRE", "NOMLARGO"]]
df_filtred["VERIFICACION"] = df_filtred.apply(lambda row: row["U##NOMBRE"] == row["NOMBRE"] == row["NOMLARGO"], axis=1)

In [85]:
df_filtred.value_counts("VERIFICACION")

VERIFICACION
False    520
True       5
Name: count, dtype: int64

In [88]:
df_ver = df_filtred[df_filtred["VERIFICACION"] == False]

In [90]:
df_ver["NOMLARGO"] = df_ver["NOMLARGO"].apply(lambda x: x.decode('ascii') if isinstance(x, bytes) else x)

/tmp/ipykernel_10337/950514676.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ver["NOMLARGO"] = df_ver["NOMLARGO"].apply(lambda x: x.decode('ascii') if isinstance(x, bytes) else x)


In [ ]:
texto_mal_codificado = "FONDO BBVA M�XICO DEUDA EMPRESAS, S.A. DE C.V. F.I.I.D."

# Decodificar el texto directamente desde utf-8
try:
    texto_decodificado = texto_mal_codificado.encode('latin1').decode('utf-8')
except UnicodeEncodeError:
    print("Error al codificar el texto con latin1.")

Error al codificar el texto con latin1.


In [ ]:
df_ver["NOMLARGO"] = df_ver["NOMLARGO"].apply(lambda x: x.upper())

In [ ]:
df_ver

In [ ]:
def nombre(df):
    